In [1]:
### import necessary libraries
import ee
import json
import geemap
import sklearn
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

ee.Authenticate()
ee.Initialize(project="ee-franciscofurey")

In [ ]:
basin = ee.FeatureCollection("WWF/HydroSHEDS/v1/Basins/hybas_7")
gcp = ee.FeatureCollection("users/ujavalgandhi/e2e/arkavathy_gcps")
s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
alos = ee.Image('JAXA/ALOS/AW3D30/V2_2')

In [ ]:
arkavathy = basin.filter(ee.Filter.eq('HYBAS_ID', 4071139640))
geometry = arkavathy.geometry()

Map = geemap.Map()

Map.centerObject(geometry)

# Definir la visualización RGB.
rgbVis = {
    'min': 0.0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2'],
}

def mask_cloud_and_shadow_sr(image: ee.Image) -> ee.Image:
    """
    Enmascara nubes y sombras en imágenes Sentinel-2 basándose en las bandas de probabilidad de nubes,
    probabilidad de sombras y la clasificación de la cobertura del suelo (SCL).
    
    Parameters:
    - image: ee.Image, imagen Sentinel-2 a enmascarar.
    
    Returns:
    - ee.Image, imagen con máscara aplicada para nubes y sombras.
    """
    cloud_prob = image.select('MSK_CLDPRB')
    cloud_shadow_prob = image.select('MSK_SNWPRB')
    cloud = cloud_prob.lt(10)  # Nubes con probabilidad menor al 10%
    scl = image.select('SCL')  # Clasificación de la cobertura del suelo
    shadow = cloud_shadow_prob.lt(3)  # Sombras con probabilidad menor al 3%
    # No usamos 'MSK_CLDCIR' ya que no está en las bandas disponibles.
    mask = cloud.And(shadow)  # Combinar máscaras de nubes y sombras
    return image.updateMask(mask)


# Filtrar la colección de imágenes por porcentaje de nubes y fecha.
filtered = s2\
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))\
    .filter(ee.Filter.date('2019-01-01', '2020-01-01'))\
    .filter(ee.Filter.bounds(geometry))\
    .map(mask_cloud_and_shadow_sr)\
    .select('B.*')

# Crear un compuesto usando la mediana.
composite = filtered.median()

Map.addLayer(composite.clip(geometry), rgbVis, 'image') # Esta línea se usa para agregar una capa al mapa en la UI de JavaScript, no es aplicable en Python.

def add_indices(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('ndvi')
    ndbi = image.normalizedDifference(['B11', 'B8']).rename('ndbi')
    mndwi = image.normalizedDifference(['B3', 'B11']).rename('mndwi')
    evi = image.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
            'NIR': image.select('B8'),
            'RED': image.select('B4'),
            'BLUE': image.select('B2')
        }).rename('EVI')
    bsi = image.expression(
        '((SWIR1 + RED) - (NIR + BLUE)) / ((SWIR1 + RED) + (NIR + BLUE))', {
            'SWIR1': image.select('B11'),
            'RED': image.select('B4'),
            'NIR': image.select('B8'),
            'BLUE': image.select('B2'),
        }).rename('BSI')
    return image.addBands(ndvi).addBands(ndbi).addBands(mndwi).addBands(bsi).addBands(evi)

composite = add_indices(composite)

# Get elevation data and slope from alos
elevation = alos.select('AVE_DSM').rename('elevation')
slope = ee.Terrain.slope(alos.select('AVE_DSM')).rename('slope')

composite = composite.addBands(elevation).addBands(slope)

def normalize(image):
    bands = image.bandNames()
    min_dict = image.reduceRegion(
        reducer=ee.Reducer.min(),
        geometry=geometry,
        scale=10,
        maxPixels=1e9,
        bestEffort=True,
        tileScale=16,
    )
    max_dict = image.reduceRegion(
        reducer=ee.Reducer.max(),
        geometry=geometry,
        scale=10,
        maxPixels=1e9,
        bestEffort=True,
        tileScale=16,
    )
    mins = ee.Image.constant(min_dict.values(bands))
    maxs = ee.Image.constant(max_dict.values(bands))
    # Normalize the image
    normalized = image.subtract(mins).divide(maxs.subtract(mins))
    return normalized

composite = normalize(composite)

# Añadir una columna aleatoria y dividir los GCPs en conjuntos de entrenamiento y validación.
gcp = gcp.randomColumn()
trainingGcp = gcp.filter(ee.Filter.lt('random', 0.6))
validationGcp = gcp.filter(ee.Filter.gte('random', 0.6))

# Superponer los puntos sobre la imagen para obtener los datos de entrenamiento.
training = composite.sampleRegions(
    collection=trainingGcp,
    properties=['landcover'],
    scale=10,
    tileScale=16
)

# Entrenar un clasificador.
classifier = ee.Classifier.smileRandomForest(50).train(
    features=training,
    classProperty='landcover',
    inputProperties=composite.bandNames()
)

# Clasificar la imagen.
classified = composite.classify(classifier)

palette = ['#cc6d8f', '#ffc107', '#1e88e5', '#004d40']
Map.addLayer(classified.clip(geometry), {'min': 0, 'max': 3, 'palette': palette}, '2019')
